# Step 4: visualization

In [1]:
import sys
import os
from os.path import join
import time
from datetime import datetime
import importlib
import numpy as np
import urllib.request
from tqdm import tqdm
from tqdm import trange
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib import patches
import plotly.graph_objs as go
from ipywidgets import interact
from ipywidgets import interactive
from ipywidgets import interactive_output
from ipywidgets import widgets
from IPython.display import display
from IPython.display import clear_output
import proplot as pplt

from tools import plotting as mplt
from tools import utils

In [2]:
mpl.rcParams['path.simplify'] = True
mpl.rcParams['path.simplify_threshold'] = 1.0
mpl.style.use('fast')
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'

## Load data 

The distribution file is a bit too large (4GB) to store on GitHub, so it needs to be downloaded when the notebook is run. This takes at least ten minutes. Binder normally alocates 2GB of memory for notebooks, but in my case it looks like they gave me 8GB; even so, it is easy to go over this limit and kill the notebook. The following 2D plots seem to work okay in isolation.

In [ ]:
url = "https://www.dropbox.com/s/a7uw6e5ef59oli5/a5d_new.mmp?dl=1"
filename = "a5d_new.mmp"
urllib.request.urlretrieve(url, filename)

## Load data 

In [ ]:
folder = '.'

In [ ]:
shape = tuple(np.loadtxt(join(folder, 'a5d_new_shape.txt')).astype(int))
shape

In [ ]:
coords = utils.load_stacked_arrays(join(folder, 'coords.npz'))
for c in coords:
    print(c.shape)

In [ ]:
f = np.memmap(join(folder, 'a5d_new.mmp'), shape=shape, dtype='float', mode='r')

In [ ]:
print(f'min(f) = {np.min(f)}')
f = np.clip(f, 0.0, None)

Flip the energy axis (to make up for previous sign error).

In [ ]:
f = f[..., ::-1]

## Plot 

In [ ]:
# Settings
dims = ["x", "xp", "y", "yp", "w"]
units = ["mm", "mrad", "mm", "mrad", "MeV"]
dims_units = [f'{d} [{u}]' for d, u in zip(dims, units)]
dim_to_int = {dim: i for i, dim in enumerate(dims)}
prof_kws = dict(lw=1.0, alpha=0.5, color='white', scale=0.15)

# Widgets
default_ind = (2, 3)  # plot on initial render
cmaps = ['viridis', 'dusk_r', 'mono_r', 'plasma']
cmap = widgets.Dropdown(options=cmaps, description='cmap')
thresh = widgets.FloatSlider(value=-5.0, min=-8.0, max=0.0, step=0.1, description='thresh', continuous_update=True)
discrete = widgets.Checkbox(value=False, description='discrete')
log = widgets.Checkbox(value=False, description='log')
contour = widgets.Checkbox(value=False, description='contour')
profiles = widgets.Checkbox(value=True, description='profiles')
scale = widgets.FloatSlider(value=0.15, min=0.0, max=1.0, step=0.01, description='scale')
dim1 = widgets.Dropdown(options=dims, index=default_ind[0], description='dim 1')
dim2 = widgets.Dropdown(options=dims, index=default_ind[1], description='dim 2')

# Sliders
sliders, range_sliders, checks = [], [], []
for k in range(5):
    slider = widgets.IntSlider(
        min=0, max=shape[k], value=shape[k]//2,
        description=dims[k], 
        continuous_update=True,
    )
    range_slider = widgets.IntRangeSlider(
        value=(0, f.shape[k]), min=0, max=f.shape[k],
        description=dims[k], 
        continuous_update=True,
    )
    for _slider in (slider, range_slider):
        slider.layout.display = 'none'
    sliders.append(slider)
    range_sliders.append(range_slider)
    checks.append(widgets.Checkbox(description=f'slice {dims[k]}'))
    
# Hide/show sliders.
def hide(button):
    for k in range(5):
        # Hide elements for dimensions being plotted.
        valid = dims[k] not in (dim1.value, dim2.value)
        disp = None if valid else 'none'
        for element in [sliders[k], range_sliders[k], checks[k]]:
            element.layout.display = disp
        # Uncheck boxes for dimensions being plotted. 
        if not valid and checks[k].value:
            checks[k].value = False
        # Make sliders respond to check boxes.
        if not checks[k].value:
            for slider in [sliders[k], range_sliders[k]]:
                slider.layout.display = 'none'
                
for element in (dim1, dim2, *checks):
    element.observe(hide, names='value')
# Initial hide
for k in range(5):
    if k in default_ind:
        checks[k].layout.display = 'none'
    for slider in [sliders[k], range_sliders[k]]:
        slider.layout.display = 'none'

In [ ]:
def update(dim1, dim2, check_x, check_xp, check_y, check_yp, check_w, 
           x, xp, y, yp, w, log, profiles, thresh, cmap):
    if (dim1 == dim2):
        return
    checks = [check_x, check_xp, check_y, check_yp, check_w]
    for dim, check in zip(dims, checks):
        if check and dim in (dim1, dim2):
            return
    axis_view = [dim_to_int[dim] for dim in (dim1, dim2)]
    axis_slice = [dim_to_int[dim] for dim, check in zip(dims, checks) if check]
    ind = [x, xp, y, yp, w]
    for k in range(5):
        if type(ind[k]) is int:
            ind[k] = (ind[k], ind[k] + 1)
    ind = [ind[k] for k in axis_slice]
    H = f[utils.make_slice(f.ndim, axis_slice, ind)]
    H = utils.project(H, axis_view)
    H_max = np.max(H)
    if H_max > 0:
        H = H / H_max
        
    fig, ax = pplt.subplots()
    mplt.plot_image(
        H, x=coords[axis_view[0]], y=coords[axis_view[1]], ax=ax, 
        norm='log' if log else None,
        frac_thresh=10.0**thresh,
        profx=profiles,
        profy=profiles,
        prof_kws=dict(lw=1.0, alpha=0.75, color='white', scale=0.15),
        cmap=cmap,
        colorbar=True,
    )
    ax.format(xlabel=dims_units[axis_view[0]], ylabel=dims_units[axis_view[1]])
    plt.show()

### 2D projection of int slice 

Slicing along each dimension is controlled by the checkboxes and sliders. The sliced distribution is projected onto dimensions `dim1` and `dim2`. 

In [ ]:
interactive(
    update, 
    dim1=dim1, dim2=dim2, 
    check_x=checks[0], check_xp=checks[1], check_y=checks[2], check_yp=checks[3], check_w=checks[4],
    x=sliders[0], xp=sliders[1], y=sliders[2], yp=sliders[3], w=sliders[4],
    log=log, profiles=profiles, thresh=thresh, cmap=cmap,
)

### 2D projection of range slice

This allows us to slice a range of indices along each dimension. Unfortunately, ipywidgets does not currently allow you to drag the center of the range slider. 

In [ ]:
interactive(
    update, 
    dim1=dim1, dim2=dim2, 
    check_x=checks[0], check_xp=checks[1], check_y=checks[2], 
    check_yp=checks[3], check_w=checks[4],
    x=range_sliders[0], xp=range_sliders[1], y=range_sliders[2], 
    yp=range_sliders[3], w=range_sliders[4],
    log=log, profiles=profiles, thresh=thresh, cmap=cmap,
)